# Traffic Reduction Scenarios
**Important**! You can alter *two different types* of controls below:
* *Policy Levers* - we have a good evidence base for what changing these things will do to travel behaviour on average
* *Top down assumptions* - we lack conclusive evidence on how to incentivise these changes. You can specify e.g. a percentage of telework or increase in cycling to test the effects, but we can't tell you how to get there.

[View the report](https://users.cs.cf.ac.uk/CooperCH/TrafRed/methodology/TrafRed%20model%20specification%201.0.pdf) for more detail on all assumptions, limitations and methods.

↓↓ *To start, **click the Play icon below**.
If you are not already signed into a Google account, you will be asked to sign in.*

In [ ]:
# (C) Cardiff University 2022 onwards. Released open source, license here: https://www.gnu.org/licenses/agpl-3.0.en.html
#@title
from ipywidgets import widgets
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import copy
plt.rcParams.update({'font.size': 8})

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

# Load census data to get urban/rural population mix
urban_rural_mix_table = pd.read_csv("https://raw.githubusercontent.com/fiftysevendegreesofrad/TrafRed/master/la%20composition.csv",index_col=0).dropna(axis=0,how="all").dropna(axis=1,how="all")
urban_rural_mix_table.index.name="local authority"
del urban_rural_mix_table["Total"]
# insert special row for all England + Wales
urban_rural_mix_table.loc["England and Wales"]=urban_rural_mix_table.sum()

# insert combined authorities
combined_authorities = '''Cambridgeshire and Peterborough,Cambridgeshire,Peterborough
Greater Manchester,Bolton,Bury,Oldham,Manchester,Rochdale,Salford,Tameside,Trafford,Wigan
Liverpool City Region,Halton,Knowsley,Liverpool,Sefton,St. Helens,Wirral
North of Tyne,Newcastle upon Tyne,North Tyneside,Northumberland
South Yorkshire,Barnsley,Doncaster,Rotherham,Sheffield
Tees Valley,Darlington,Hartlepool,Stockton-on-Tees,Middlesbrough,Redcar and Cleveland
West Midlands,Birmingham,Coventry,Dudley,Sandwell,Solihull,Walsall,Wolverhampton
West of England,Bath and North East Somerset,Bristol: City of,South Gloucestershire
West Yorkshire,Bradford,Calderdale,Kirklees,Leeds,Wakefield
North East,County Durham,Gateshead,South Tyneside,Sunderland'''
combined_authorities = [[y.replace(":",",") for y in x.split(",")] for x in combined_authorities.replace('\xa0',' ').split("\n")]
for ca in combined_authorities:
  combined_name = ca[0]
  constituent_las = ca[1:]
  for cla in constituent_las:
    if not cla in urban_rural_mix_table.index:
      assert "," not in cla
      print("Constitutent LA not found:",cla)
  urban_rural_mix_table.loc[combined_name] = urban_rural_mix_table.loc[urban_rural_mix_table.index.isin(constituent_las)].sum()

ruralcats = ["Urban Conurbation",
            "Urban City and Town",
            "Rural Town and Fringe",
            "Rural Village, Hamlet and Isolated Dwelling"]

def load(local_authority):
  urban_rural_mix = urban_rural_mix_table.loc[local_authority]

  # Load NTS9907 to get total distance by purpose
  national_average_distance_by_rural_and_purpose = pd.read_csv("https://raw.githubusercontent.com/fiftysevendegreesofrad/TrafRed/master/nts9907.csv",index_col=0).dropna(axis=0,how="all").dropna(axis=1,how="all")
  total_distance_by_purpose = pd.DataFrame({col:national_average_distance_by_rural_and_purpose[col]*urban_rural_mix for col in national_average_distance_by_rural_and_purpose.columns})
  total_distance_by_purpose = total_distance_by_purpose.stack().to_frame()
  total_distance_by_purpose.index.names=["ruralcat","purpose"]
  total_distance_by_purpose.columns=["model distance py"]
  # Load figures from NTSQ01008 to get distance bands for each rural category
  national_distance_bands_by_rural = pd.read_csv("https://raw.githubusercontent.com/fiftysevendegreesofrad/TrafRed/master/NTSQ01008.csv",index_col=0).dropna(axis=0,how="all").dropna(axis=1,how="all").apply(pd.to_numeric)
  totals_per_ruralcat = national_distance_bands_by_rural.sum(axis=1)
  national_distance_bands_by_rural = pd.DataFrame({float(col):national_distance_bands_by_rural[col]/totals_per_ruralcat for col in national_distance_bands_by_rural.columns})
  national_distance_bands_by_rural = national_distance_bands_by_rural.stack().to_frame()
  national_distance_bands_by_rural.columns = ["prop ruralcat dist in band"]
  national_distance_bands_by_rural.index.names = ["ruralcat","Distance"]
  # Multiply total_distance_by_purpose by national_distance_bands_by_rural to break down total distance into trip length bands
  national_distance_bands_by_rural = national_distance_bands_by_rural.reset_index()
  national_distance_bands_by_rural = national_distance_bands_by_rural.set_index("ruralcat")
  total_distance_by_purpose_rural = total_distance_by_purpose.join(national_distance_bands_by_rural,on="ruralcat")
  total_distance_by_purpose_rural["model distance py"]*=total_distance_by_purpose_rural["prop ruralcat dist in band"]
  del total_distance_by_purpose_rural["prop ruralcat dist in band"]

  # Load NTS0308 to get modal split for each distance band
  mode_split = pd.read_csv("https://raw.githubusercontent.com/fiftysevendegreesofrad/TrafRed/master/nts0308_2019_trips_pppy.csv").dropna(axis=0,how="all").dropna(axis=1,how="all").set_index("Main mode").stack().to_frame()
  mode_split.columns = ["Trips pppy"]
  mode_split.index.set_names(["Orig Mode","Distance"],inplace=True)
  mode_split = mode_split.reset_index()
  mode_split["Distance"] = pd.to_numeric(mode_split["Distance"])
  mode_split.loc[mode_split.Distance==150,"Distance"]=252 # see https://github.com/fiftysevendegreesofrad/TrafRedPrivate/issues/2
  mode_split["Distance pppy"]=mode_split["Trips pppy"]*mode_split["Distance"]
  del mode_split["Trips pppy"]

  # Aggregate set of travel modes from source data into the set we want to use
  mode_split["Mode"] = None # create blank column
  my_aggregation = {"Walk":["Walk"],
                    "Bicycle":["Bicycle"],
                    "Car":["Car / van driver","Motorcycle","Taxi / minicab"],
                    "Car passenger":["Car / van passenger"],
                    "Bus":["Bus in London","Other local bus","Non-local bus","Other private transport"],
                    "Rail":["London Underground","Surface Rail"]}
  for new_mode,old_modes in my_aggregation.items():
    for old_mode in old_modes:
      mode_split.loc[mode_split["Orig Mode"]==old_mode,"Mode"] = new_mode
  del mode_split["Orig Mode"]
  mode_split = mode_split.groupby(["Mode","Distance"]).sum(numeric_only=True).unstack()

  # Normalize per distance band
  mode_split /= mode_split.sum(axis=0)
  mode_split = mode_split.rename({"Distance pppy":"prop distance band in mode"},axis=1).stack()

  mode_split = mode_split.reset_index().set_index("Distance")
  baseline = total_distance_by_purpose_rural.join(mode_split,on="Distance")
  baseline["model distance py"] *= baseline["prop distance band in mode"]
  del baseline["prop distance band in mode"]
  baseline = baseline.reset_index()
  return baseline

def load_global(local_authority):
  global global_baseline
  global_baseline = load(local_authority)

load_global("England and Wales")

def graphme(*args):
  pass

mode_to_colour = {"Car":"red",
                  "Car passenger":"palevioletred",
                  "Bicycle":"teal",
                  "Bus":"blue",
                  "Rail":"k",
                  "Walk":"darkgreen",
                  "London":"k",
                  "Urban Conurbation":"grey",
                  "Rural Town and Fringe":"orange",
                  "Rural Village, Hamlet and Isolated Dwelling":"forestgreen",
                  "Urban City and Town":"b",
                  "Total":"#111111"
                  }

baseline_speed_local_a_road = 24
baseline_speeds_dict = {"speed_local":13,
                   "speed_local_a_road_rural":baseline_speed_local_a_road,
                   "speed_local_a_road_urban":baseline_speed_local_a_road,
                   "speed_strategic":58.5}
baseline_speeds = Struct(**baseline_speeds_dict)

time_to_strategic_net_baseline = {"Urban Conurbation":25/60,
                                  "Urban City and Town":25/60,
                                  "Rural Town and Fringe":50/60,
                                  "Rural Village, Hamlet and Isolated Dwelling":50/60}

time_to_local_a_road_minutes = 5

distance_to_strategic_net = {}
for cat,time in time_to_strategic_net_baseline.items():
  # assume x mins at local speed and remainder at local a-road speed
  assert time>time_to_local_a_road_minutes/60
  assert baseline_speeds.speed_local_a_road_rural == baseline_speeds.speed_local_a_road_urban
  distance_to_strategic_net[cat] = time_to_local_a_road_minutes/60*baseline_speeds.speed_local + (time-time_to_local_a_road_minutes/60)*baseline_speeds.speed_local_a_road_rural

distance_to_local_a_road = time_to_local_a_road_minutes/60*baseline_speeds.speed_local

def journey_time(from_cat,to_cat,distance,p):
  remaining_distance = distance

  # subtract end stages
  first_stage_distance = distance_to_local_a_road
  first_stage_time = first_stage_distance/p.speed_local
  end_stage_distance = distance_to_local_a_road
  end_stage_time = end_stage_distance/p.speed_local

  if (remaining_distance - first_stage_distance - end_stage_distance <= 0):
    return remaining_distance / p.speed_local

  remaining_distance -= first_stage_distance
  remaining_distance -= end_stage_distance
  cumulative_time = first_stage_time + end_stage_time

  # subtract second and penultimate stages
  ruralcat_local_a_road_speeds = {"Urban Conurbation":p.speed_local_a_road_urban,
                                  "Urban City and Town":p.speed_local_a_road_urban,
                                  "Rural Town and Fringe":p.speed_local_a_road_rural,
                                  "Rural Village, Hamlet and Isolated Dwelling":p.speed_local_a_road_rural}

  second_stage_distance = distance_to_strategic_net[from_cat]-first_stage_distance
  penultimate_stage_distance = distance_to_strategic_net[to_cat]-end_stage_distance
  second_stage_time = second_stage_distance / ruralcat_local_a_road_speeds[from_cat]
  penultimate_stage_time = penultimate_stage_distance / ruralcat_local_a_road_speeds[to_cat]
  if (remaining_distance - second_stage_distance - penultimate_stage_distance <= 0):
    # assume proportionate distance in each band scaled down to total remaining_distance
    second_stage_dist_proportion = second_stage_distance/(second_stage_distance+penultimate_stage_distance)
    penultimate_stage_dist_proportion = 1-second_stage_dist_proportion
    second_stage_time = remaining_distance*second_stage_dist_proportion / ruralcat_local_a_road_speeds[from_cat]
    penultimate_stage_time = remaining_distance*penultimate_stage_dist_proportion / ruralcat_local_a_road_speeds[to_cat]
    return cumulative_time + second_stage_time+penultimate_stage_time

  remaining_distance -= second_stage_distance
  remaining_distance -= penultimate_stage_distance
  cumulative_time += second_stage_time + penultimate_stage_time

  # remainder of journey at strategic speed
  remaining_time = remaining_distance/p.speed_strategic
  return cumulative_time + remaining_time

threshold_settlement_radius = {"Urban Conurbation":5,
                                  "Urban City and Town":5,
                                  "Rural Town and Fringe":1,
                                  "Rural Village, Hamlet and Isolated Dwelling":0}

def mean_journey_time(row,p):
  if row.Distance<threshold_settlement_radius[row.ruralcat]:
    # assume all trips stay in same ruralcat
    return journey_time(row.ruralcat,row.ruralcat,row.Distance,p)
  else:
    times = []
    proportions = []
    for rc in ruralcats:
      proportions.append(row[rc])
      times.append(journey_time(row.ruralcat,rc,row.Distance,p))
    return (np.array(times)*np.array(proportions)).sum()

length_slices = [1,2,5,10,25,50,100]

def length_under(miles):
  assert miles in length_slices
  return lambda x: x<miles

def length_over(miles):
  assert miles in length_slices
  return lambda x: x>=miles

def all_trips():
  return lambda x: True

# bus model assumes constant fare per mile by default. while this works in aggregate, it looks rather low for short trips, so it doesn't make sense to disaggregate these
bus_fare_cap_length_slices = [x for x in length_slices if x>=10]

bus_fare_cap_to_predicate = {"no trips (don't apply fare cap)":lambda x: False,
                             **{f"trips under {d} miles":length_under(d) for d in bus_fare_cap_length_slices},
                             "all trips":all_trips()}

car_price_elasticities = [(length_over(10),{"Commute":-0.31,
                                "Shop/Education":-0.31,
                                "Business":-0.11,
                                "Leisure":-0.63}),
                (length_under(10),{"Commute":-0.16,
                          "Shop/Education":-0.16,
                          "Business":-0.10,
                          "Leisure":-0.33})]

car_time_elasticities = [(length_over(10),{"Commute":-0.98,
                                "Shop/Education":-0.98,
                                "Business":-0.75,
                                "Leisure":-0.99}),
                (length_under(10),{"Commute":-0.79,
                          "Shop/Education":-0.79,
                          "Business":-0.60,
                          "Leisure":-0.80})]

# tuples show (own-ped,bus-car-cross-ped)
bus_price_elasticities = [(length_over(10),{"Commute":(-.63,.04),
                                "Shop/Education":(-.63,.04),
                                "Business":(-.63,.02),
                                "Leisure":(-.91,.04)}),
                (length_under(10),{"Commute":(-.44,.04),
                          "Shop/Education":(-.63,.04),
                          "Business":(-.63,.02),
                          "Leisure":(-(.74+.28)/2,.04)})]

# tuples show (own-ped,rail-car-cross-ped)
rail_price_elasticities = [(length_over(10),{"Commute":(-(.72+.9)/2,.18),
                                "Shop/Education":(-(.72+.9)/2,.18),
                                "Business":(-(.64+.75)/2,.12),
                                "Leisure":(-(1.03+1.21)/2,.18)}),
                (length_under(10),{"Commute":(-(.59+.77)/2,.05),
                          "Shop/Education":(-(.59+.77)/2,.05),
                          "Business":(-(.57+.75)/2,.03),
                          "Leisure":(-(1.35+1.23)/2,.05)})]

bus_headway_elasticities = [(length_under(10),{"Commute":-.34,
                                "Shop/Education":-.34,
                                "Business":-.34,
                                "Leisure":-.28}),
                (length_over(10),{"Commute":-.36,
                          "Shop/Education":-.36,
                          "Business":-.36,
                          "Leisure":-.36})]

bus_time_elasticities = [(length_under(10),{"Commute":-.5,
                                "Shop/Education":-.5,
                                "Business":-.5,
                                "Leisure":-.4}),
                (length_over(10),{"Commute":-.84,
                          "Shop/Education":-.84,
                          "Business":-.84,
                          "Leisure":-.84})]

rail_headway_elasticities = [(length_under(25),-(.29+.34+.21+.24)/4),
                             (length_over(25),-(.19+.23+.14+.16)/4)]

# diversion factors take midpoint values from Dunkerley 2018
diversion_factors_init = {"Car":[(length_under(10),{"Bus":0.3,"Rail":(0.05+0.2+0.1+0.35)/4,"Bicycle":0.05,"Walk":0.1}),
                        (length_over(10),{"Bus":0.09,"Rail":0.65,"Bicycle":0,"Walk":0})],
                        "Car passenger":[(length_under(10),{"Bus":0.3,"Rail":(0.05+0.2+0.1+0.35)/4,"Bicycle":0.05,"Walk":0.1}),
                        (length_over(10),{"Bus":0.09,"Rail":0.65,"Bicycle":0,"Walk":0})],
                     "Rail":[(length_under(10),{"Car":(0.3+0.45)/2,"Bus":(.25+.4)/2,"Bicycle":.05,"Walk":.025}),
                          (length_over(10),{"Car":(0.4+.55)/2,"Bus":.15,"Bicycle":0,"Walk":0})],
                     "Bus":[(length_under(10),{"Car":(.2+.35)/2,"Rail":(.05+.35)/2,"Bicycle":.06,"Walk":.2}),
                          (length_over(10),{"Car":(.2+.35)/2,"Rail":(.45+.65)/2,"Bicycle":0,"Walk":0})],
                     "Bicycle":[(length_under(10),{"Car":.19,"Rail":.13,"Bus":.19,"Walk":.19}),
                          (length_over(10),{"Car":.19,"Rail":.13,"Bus":.19,"Walk":.19})]}

ruralcat_car_trip_matrix = pd.read_json('{"(\'From\', \'Urban Conurbation\')":{"Urban Conurbation":0.7914590828,"Urban City and Town":0.1553595379,"Rural Town and Fringe":0.035603442,"Rural Village, Hamlet and Isolated Dwelling":0.0175779373},"(\'From\', \'Urban City and Town\')":{"Urban Conurbation":0.0909440482,"Urban City and Town":0.6989070896,"Rural Town and Fringe":0.1078179074,"Rural Village, Hamlet and Isolated Dwelling":0.1023309548},"(\'From\', \'Rural Town and Fringe\')":{"Urban Conurbation":0.1215115972,"Urban City and Town":0.5406394691,"Rural Town and Fringe":0.170763398,"Rural Village, Hamlet and Isolated Dwelling":0.1670855356},"(\'From\', \'Rural Village, Hamlet and Isolated Dwelling\')":{"Urban Conurbation":0.0556432085,"Urban City and Town":0.5186550523,"Rural Town and Fringe":0.1656029408,"Rural Village, Hamlet and Isolated Dwelling":0.2600987983}}')
ruralcat_car_trip_matrix.columns = [eval(x)[1] for x in list(ruralcat_car_trip_matrix.columns)] # remove 'from' from weird json nested column strings
ruralcat_car_trip_matrix = ruralcat_car_trip_matrix.transpose() # now 'from' is rows

purposes = ["Commute","Shop/Education","Business","Leisure"]

def car_mode(df):
  return (df.Mode=="Car")|(df.Mode=="Car passenger")
def pt_mode(df):
  return (df.Mode=="Rail")|(df.Mode=="Bus")
def rc_match(df,rc):
  if rc=="urban":
    return (df.ruralcat.isin(ruralcats[0:2]))
  else:
    assert rc=="rural"
    return (df.ruralcat.isin(ruralcats[2:4]))

# costs based on average fuel costs, based on HMRC advisory rates averaging petrol/diesel
baseline_car_fuel_costs_per_mile = 0.19

def scenario_combined_activity_model(params):
  warnings = []

  percent_to_factor_conversions = [("car_cost_factor","car_cost_increase_percent"),
                                   ("bus_frequency_increase","bus_frequency_increase_percent"),
                                   ("bus_speed_increase","bus_speed_increase_percent"),
                                   ("rail_frequency_increase","rail_frequency_increase_percent"),
                                   ("bicycle_increase_factor","bicycle_increase_percent")]

  for factor,percent_change in percent_to_factor_conversions:
    params[factor] = (params[percent_change]+100)/100
    del params[percent_change]

  for factor,percent_change in [("rail_fare_discount","rail_fare_discount_percent"),
                                ("bus_fare_discount","bus_fare_discount_percent")]:
    params[factor] = params[percent_change]/100
    del params[percent_change]

  p = Struct(**params)

  ruralcats_sum = p.baseline.groupby("ruralcat")["model distance py"].sum(numeric_only=True)
  ruralcats_in_this_la = list(ruralcats_sum[ruralcats_sum>0].index)

  result = p.baseline.copy()
  result["scenario distance py"]=result["model distance py"]

  # dynamically add diversion factor from car driver to car passenger (table already includes car passenger to other modes)
  diversion_factors = diversion_factors_init.copy()
  for length_pred,impacted_mode_to_df in diversion_factors["Car"]:
    if "Car passenger" in impacted_mode_to_df:
      del impacted_mode_to_df["Car passenger"]
    total_car_driver_df_this_length = sum([df for _,df in impacted_mode_to_df.items()])
    no_travel_or_passenger_df = 1-total_car_driver_df_this_length
    assert no_travel_or_passenger_df > 0
    impacted_mode_to_df["Car passenger"] = p.unaccounted_driver_to_passenger_proportion * no_travel_or_passenger_df

  # apply car club model - 35% reduction in veh km for members
  ruralcats_car_club_applies = ruralcats[0:ruralcats.index(p.most_rural_car_club_category)+1]
  for rc in ruralcats_car_club_applies:
    result.loc[car_mode(result) & (result.ruralcat==rc),"scenario distance py"] *= (1-.35*p.car_club_uptake_percent/100)

  if p.car_club_uptake_percent>0 and not(set(ruralcats_car_club_applies)&set(ruralcats_in_this_la)):
    warnings.append("Car club uptake has been set, but does not apply to any urban/rural category in this local authority")

  # apply integrated transport model - 14% uplift in bus and rail.
  # But we actually increase by more on both bus and rail to compensate for diversion factor modelling which assumes extra use on bus would be stolen from rail, and vice versa
  if p.integrated_transport:
    result.loc[result.Mode=="Bus","scenario distance py"] *= 1.20
    result.loc[result.Mode=="Rail","scenario distance py"] *= 1.18

  # apply car price model
  baseline_car_costs_per_trip_pair = 0

  # note a trip in NTS is 1 way, so costs such as congestion charge/parking are spread over 2 trips
  ruralcats_congestion_charge_applies = ruralcats[0:ruralcats.index(p.most_rural_cc_category)+1]
  # merge inter-ruralcat trip matrix with trip table, hence adding columns for proportion of trips going to each ruralcat
  result = result.merge(ruralcat_car_trip_matrix,left_on="ruralcat",right_index=True)
  result["trip_pair_fixed_charge"] = 0
  for rc in ruralcats_congestion_charge_applies:
    # add congestion times proportion of trips to that category
    result.loc[car_mode(result) & p.length_predicate(result.Distance),"trip_pair_fixed_charge"] += p.congestion_charge * result.loc[:,rc]
  for rc in ruralcats_congestion_charge_applies:
    # set (don't add) congestion charge on all trips in that category
    result.loc[car_mode(result) & (result.ruralcat==rc) & p.length_predicate(result.Distance),"trip_pair_fixed_charge"] = p.congestion_charge

  if p.congestion_charge>0 and not(set(ruralcats_congestion_charge_applies)&set(ruralcats_in_this_la)):
    warnings.append("A congestion charge has been set, but does not apply to any urban/rural category in this local authority")

  # add parking charge by purpose and rural, multiplied by proportion of trips with DESTINATIONS in relevant ruralcat
  for purpose in purposes:
    for parking_charge,ruralcats_parking_charge_applies in [(params[f"parking_charge_{purpose}_urban"],ruralcats[0:2]),
                                                            (params[f"parking_charge_{purpose}_rural"],ruralcats[2:4])]:
      for rc in ruralcats_parking_charge_applies:
        result.loc[car_mode(result) & (result.purpose==purpose) & p.length_predicate(result.Distance),"trip_pair_fixed_charge"] += parking_charge*result.loc[:,rc]

  # note these are not adjusted for occupancy. It doesn't matter, as we consider only the ratio between them below.
  result.loc[car_mode(result),"baseline_trip_car_cost_per_vehicle"] = result.Distance * baseline_car_fuel_costs_per_mile + 0.5*baseline_car_costs_per_trip_pair
  result.loc[car_mode(result),"scenario_trip_car_cost_per_vehicle"] = result.Distance * baseline_car_fuel_costs_per_mile*p.car_cost_factor + 0.5*(baseline_car_costs_per_trip_pair + result["trip_pair_fixed_charge"])

  for elasticity_length_predicate,elasticities in car_price_elasticities:
    for purpose,elasticity in elasticities.items():
      def filter(df):
        return p.length_predicate(df.Distance) & elasticity_length_predicate(df.Distance) & (df.purpose==purpose)

      result.loc[car_mode(result) & filter(result),"scenario distance py"] *= ((result.scenario_trip_car_cost_per_vehicle/result.baseline_trip_car_cost_per_vehicle)**elasticity)

  # nb these are not not total running costs. this is consistent with our source of elasticity data and also likely with short term behaviour
  car_extra_cost_per_mile = (p.car_cost_factor-1)*baseline_car_fuel_costs_per_mile

  # apply car time model
  result.loc[car_mode(result),"baseline car time"] = result.apply(lambda row: mean_journey_time(row,baseline_speeds),axis=1)
  result.loc[car_mode(result),"scenario car time"] = result.apply(lambda row: mean_journey_time(row,p),axis=1)
  result.loc[car_mode(result),"car time factor"] = result.loc[car_mode(result),"scenario car time"] / result.loc[car_mode(result),"baseline car time"]

  for elasticity_length_predicate,elasticities in car_time_elasticities:
    for purpose,elasticity in elasticities.items():
      def filter(df):
        return p.length_predicate(df.Distance) & elasticity_length_predicate(df.Distance) & (df.purpose==purpose)
      result.loc[car_mode(result) & filter(result),"scenario distance py"] *= (result.loc[:,"car time factor"]**elasticity)

  # Get average current bus trip length and hence fare per mile
  buses = p.baseline[p.baseline.Mode=="Bus"].groupby("Distance").sum(numeric_only=True)["model distance py"].to_frame()
  buses["ntrips"]=buses["model distance py"]/buses.index
  current_average_bus_trip_length = buses["model distance py"].sum()/buses["ntrips"].sum()
  bus_fare_per_mile = 1.43/current_average_bus_trip_length

  # apply bus subsidy model
  bus_cost_factor = 1-p.bus_fare_discount
  bus_fare_cap_predicate = bus_fare_cap_to_predicate[p.bus_fare_cap_applies]

  result.loc[(result.Mode=="Bus"),"baseline bus fare per trip"] = result.Distance * bus_fare_per_mile
  result.loc[(result.Mode=="Bus"),"scenario bus fare per trip"] = result["baseline bus fare per trip"] * bus_cost_factor
  result.loc[(result.Mode=="Bus") & (bus_fare_cap_predicate(result.Distance)),"scenario bus fare per trip"] = result.loc[:,"scenario bus fare per trip"].clip(upper = p.bus_fare_cap)
  result["bus cost factor after cap"] = result["scenario bus fare per trip"]/result["baseline bus fare per trip"]

  smallest_bus_cost_factor_we_can_model = 0.25
  largest_bus_discount_we_can_model_percent = 100*(1-smallest_bus_cost_factor_we_can_model)
  if any(result["bus cost factor after cap"]<smallest_bus_cost_factor_we_can_model):
    warnings.append(f"Some bus fares are reduced by more than {largest_bus_discount_we_can_model_percent:.0f}%, due to the fare cap. We cannot reliably model this.")
    warnings.append(f"We recommend increasing the fare cap, or reducing the trip length to which it replies. Until then, DO NOT TRUST THESE RESULTS.")

  for elasticity_length_predicate,elasticities in bus_price_elasticities:
    for purpose,(elasticity,_) in elasticities.items():
      assert elasticity<0
      def filter(df):
        return p.length_predicate(df.Distance) & elasticity_length_predicate(df.Distance) & (df.purpose==purpose)
      result.loc[(result.Mode=="Bus") & filter(result),"scenario distance py"] *= (result["bus cost factor after cap"]**elasticity)

  # apply rail subsidy model
  rail_cost_factor = 1-p.rail_fare_discount
  for elasticity_length_predicate,elasticities in rail_price_elasticities:
    for purpose,(elasticity,_) in elasticities.items():
      assert elasticity<0
      def filter(df):
        return p.length_predicate(df.Distance) & elasticity_length_predicate(df.Distance) & (df.purpose==purpose)
      result.loc[(result.Mode=="Rail") & filter(result),"scenario distance py"] *= (rail_cost_factor**elasticity)

  # apply bus frequency model
  bus_headway_factor = 1/p.bus_frequency_increase
  for elasticity_length_predicate,elasticities in bus_headway_elasticities:
    for purpose,elasticity in elasticities.items():
      def filter(df):
        return p.length_predicate(df.Distance) & elasticity_length_predicate(df.Distance) & (df.purpose==purpose)
      result.loc[(result.Mode=="Bus") & filter(result),"scenario distance py"] *= (bus_headway_factor**elasticity)

  # apply bus time model
  bus_time_factor = 1/p.bus_speed_increase
  for elasticity_length_predicate,elasticities in bus_time_elasticities:
    for purpose,elasticity in elasticities.items():
      def filter(df):
        return p.length_predicate(df.Distance) & elasticity_length_predicate(df.Distance) & (df.purpose==purpose)
      result.loc[(result.Mode=="Bus") & filter(result),"scenario distance py"] *= (bus_time_factor**elasticity)

  # apply rail frequency model
  rail_headway_factor = 1/p.rail_frequency_increase
  for elasticity_length_predicate,elasticity in rail_headway_elasticities:
    def filter(df):
      return p.length_predicate(df.Distance) & elasticity_length_predicate(df.Distance)
    result.loc[(result.Mode=="Rail") & filter(result),"scenario distance py"] *= (rail_headway_factor**elasticity)

  # apply bicycle uplift assumption
  result.loc[(result.Mode=="Bicycle"),"scenario distance py"] *= p.bicycle_increase_factor

  # apply DFs
  result["scenario distance py post-df"]=result["scenario distance py"]
  result = result.reset_index().drop("index",axis=1).set_index(["Mode","ruralcat","Distance","purpose"]).unstack(0).reset_index()
  car_proportion_passengers = result.loc[:,("scenario distance py","Car passenger")] / (result.loc[:,("scenario distance py","Car")]+result.loc[:,("scenario distance py","Car passenger")])
  car_proportion_passengers[np.isnan(car_proportion_passengers)] = 1.6/2.6 # where there is no basis to change the ratio, set average from tag data book a1.3.3
  for intervention_mode,dfs in diversion_factors.items():
    change_in_intervention_mode = result.loc[:,("scenario distance py",intervention_mode)] - result.loc[:,("model distance py",intervention_mode)]
    for df_length_predicate,impacted_mode_to_df in dfs:
      for impacted_mode,df in impacted_mode_to_df.items():
        if intervention_mode != "Car":
          # We allow diversion from Car to Car passenger, but for other modes we assume this isn't separately quantified, hence the need for splits below
          assert impacted_mode != "Car passenger"
        assert intervention_mode != impacted_mode
        if impacted_mode=="Car":
          # split impacts of other mode improvements between car drivers and passengers
          result.loc[df_length_predicate(result.Distance) & p.length_predicate(result.Distance),("scenario distance py post-df","Car")] -= change_in_intervention_mode*df * (1-car_proportion_passengers)
          result.loc[df_length_predicate(result.Distance) & p.length_predicate(result.Distance),("scenario distance py post-df","Car passenger")] -= change_in_intervention_mode*df * car_proportion_passengers
        else:
          result.loc[df_length_predicate(result.Distance) & p.length_predicate(result.Distance),("scenario distance py post-df",impacted_mode)] -= change_in_intervention_mode*df
  del car_proportion_passengers

  # apply minimum car occupancy
  # 1. compute occupancy (don't use proportion_passengers as it has changed)
  car_totals = result.loc[:,("scenario distance py post-df","Car")]+result.loc[:,("scenario distance py post-df","Car passenger")]
  car_occupancy = car_totals / result.loc[:,("scenario distance py post-df","Car")]
  car_occupancy[np.isnan(car_occupancy)] = 1.6 # where there is no basis to change the ratio, set average from tag data book a1.3.3
  # 2. recompute car and passenger cols where occupancy is below minimum
  result.loc[car_occupancy<p.min_average_car_occupancy,("scenario distance py post-df","Car")] = car_totals / p.min_average_car_occupancy
  result.loc[car_occupancy<p.min_average_car_occupancy,("scenario distance py post-df","Car passenger")] = car_totals - result.loc[:,("scenario distance py post-df","Car")]

  # rename columns
  result = result.set_index(["Distance","purpose","ruralcat"]).stack().reset_index()
  result.loc[:,"scenario distance pre-df"] = result.loc[:,"scenario distance py"]
  result.loc[:,"scenario distance py"] = result.loc[:,"scenario distance py post-df"]
  del result["scenario distance py post-df"]

  # truncate negative results that can arise from diversion factors
  result.loc[(result["scenario distance py"]<0),"scenario distance py"] = 0

  # apply ebike uptake - 20% reduction in OVERALL car and PT km for members, but taken from short trips means we want 47% reduction in <10 mile bands, and 16% in the 17.5 mile band
  result["ebike reduction"] = 0
  for rc in ["rural","urban"]:
    result.loc[(car_mode(result)|pt_mode(result)) & (result.Distance<10) & rc_match(result,rc),"ebike reduction"] = result["scenario distance py"] * .47*getattr(p,f"ebike_uptake_{rc}_percent")/100
    result.loc[(car_mode(result)|pt_mode(result)) & (result.Distance==17.5) & rc_match(result,rc),"ebike reduction"] = result["scenario distance py"] * .16*getattr(p,f"ebike_uptake_{rc}_percent")/100
  result["scenario distance py"] -= result["ebike reduction"]
  ebike_gain = result.groupby(["Distance","purpose","ruralcat"])["ebike reduction"].sum(numeric_only=True)
  result = result.set_index(["Distance","purpose","ruralcat"])
  result.loc[result.Mode=="Bicycle","scenario distance py"] += ebike_gain
  result = result.reset_index()

  # apply teleworking reduction to commute. 70% uptake from the companies that support it
  result.loc[result.purpose=="Commute","scenario distance py"] *= (1-p.wfh_uptake_percent/100*.7)

  # compute revenue
  result["Revenue"]=0
  result["Expenditure"]=0
  # Car charges apply per vehicle, not per passenger
  result.loc[(result.Mode=="Car") & p.length_predicate(result.Distance),"Revenue"] += result["scenario distance py"]*car_extra_cost_per_mile
  result.loc[(result.Mode=="Car") & p.length_predicate(result.Distance),"Revenue"] += 0.5*result["trip_pair_fixed_charge"]*result["scenario distance py"]/result.Distance # i.e. charge times number of trips

  # bus subsidies
  gdp_inflation_2015_2019 = 116.89/108.27
  bus_subsidy_per_mile = 0.054/current_average_bus_trip_length*gdp_inflation_2015_2019

  result["occupancy factor"]=1
  if p.adjust_bus_costs_for_occupancy:
    result.loc[(result.Mode=="Bus"),"occupancy factor"] *= result["model distance py"]/result["scenario distance py"]*p.bus_frequency_increase

  result.loc[(result.Mode=="Bus") & p.length_predicate(result.Distance),"Expenditure"] = (- ( (p.bus_fare_discount*bus_fare_per_mile + bus_subsidy_per_mile) * result["occupancy factor"]*result["scenario distance py"]
                                                      -bus_subsidy_per_mile*result["model distance py"] )).clip(upper=0) # subtract baseline expenditure
  # rail fares
  rail_fare_per_mile = 0.1547/5*8
  rail_subsidy_per_mile = .056 * gdp_inflation_2015_2019
  if p.adjust_rail_costs_for_occupancy:
    result.loc[(result.Mode=="Rail"),"occupancy factor"] *= result["model distance py"]/result["scenario distance py"]*p.rail_frequency_increase

  result.loc[(result.Mode=="Rail"),"Revenue"] = 0
  result.loc[(result.Mode=="Rail") & p.length_predicate(result.Distance),"Expenditure"] = (- ( (p.rail_fare_discount*rail_fare_per_mile + rail_subsidy_per_mile) * result["occupancy factor"]*result["scenario distance py"]
                                                       -rail_subsidy_per_mile*result["model distance py"] )).clip(upper=0) # subtract baseline expenditure

  #display(result)
  return Struct(**{"result":result,"warnings":warnings})

def pkm_barplot(df,baseline,groupfield):
  df["ruralcat"] = pd.Categorical(df["ruralcat"], ruralcats)
  baseline["ruralcat"] = pd.Categorical(baseline["ruralcat"], ruralcats)

  baseline_for_plot = baseline.groupby([groupfield,"Mode"]).sum(numeric_only=True).unstack()["model distance py"]/1e9
  df = df.groupby([groupfield,"Mode"]).sum(numeric_only=True).unstack()["scenario distance py"]/1e9


  fig, ax = plt.subplots(1,2,sharey=True,figsize=(10,3))
  if df.columns.nlevels==2:
    df = df.droplevel(0,axis=1)
  for c in df.columns:
    if df[c].sum()==0:
      del df[c]
  df = df[df.sum(axis=1)>0]
  df.plot.bar(ax=ax[1],stacked=True,color=[mode_to_colour[x] for x in df.columns])
  baseline_for_plot.plot.bar(ax=ax[0],stacked=True,color=[mode_to_colour[x] for x in baseline_for_plot.columns])
  xlabel = "Trip length (miles)" if groupfield=="Distance" else ""
  ax[0].set_xlabel(xlabel)
  ax[1].set_xlabel(xlabel)
  ax[0].set_ylabel("Passenger miles per annum (billions)")
  ax[0].set_title("Baseline")
  ax[1].set_title("Scenario")
  plt.show()

def create_gui():
  controls = {
    "length_filter":widgets.Dropdown(
        options=['All trips', 'Trips over', 'Trips under'],description="Apply to"
    ),
    "length_slice":widgets.Dropdown(options=length_slices),
    "car_cost_increase_percent":widgets.IntSlider(value=0,min=0,max=100,step=1,continuous_update=False),
    "congestion_charge":widgets.FloatSlider(value=0,min=0,max=15,step=.5,continuous_update=False),
    "most_rural_cc_category":widgets.Dropdown(options=ruralcats),
    "bus_fare_discount_percent":widgets.IntSlider(value=0,min=0,max=75,step=1,continuous_update=False),
    "bus_fare_cap":widgets.FloatSlider(value=2,min=1,max=5,step=.1,continuous_update=False),
    "bus_fare_cap_applies":widgets.Dropdown(options=bus_fare_cap_to_predicate.keys()),
    "bus_frequency_increase_percent":widgets.IntSlider(value=0,min=0,max=300,step=10,continuous_update=False,layout=widgets.Layout(width='200px')),
    "bus_speed_increase_percent":widgets.IntSlider(value=0,min=0,max=30,step=1,continuous_update=False,layout=widgets.Layout(width='200px')),
    "rail_fare_discount_percent":widgets.IntSlider(value=0,min=0,max=75,step=1,continuous_update=False),
    "rail_frequency_increase_percent":widgets.IntSlider(value=0,min=0,max=300,step=10,continuous_update=False,layout=widgets.Layout(width='200px')),
    "adjust_bus_costs_for_occupancy":widgets.Checkbox(description="bus"),
    "adjust_rail_costs_for_occupancy":widgets.Checkbox(description="rail"),
    "unaccounted_driver_to_passenger_proportion":widgets.FloatSlider(value=0,min=0,max=1,step=0.01,continuous_update=False,layout=widgets.Layout(width='200px')),
    "min_average_car_occupancy":widgets.FloatSlider(value=1,min=1,max=3,step=.05,continuous_update=False,layout=widgets.Layout(width='200px')),
    "most_rural_car_club_category":widgets.Dropdown(options=ruralcats),
    "car_club_uptake_percent":widgets.IntSlider(value=0,min=0,max=100,step=1,continuous_update=False),
    "ebike_uptake_urban_percent":widgets.IntSlider(value=0,min=0,max=50,step=1,continuous_update=False,layout=widgets.Layout(width='200px')),
    "ebike_uptake_rural_percent":widgets.IntSlider(value=0,min=0,max=50,step=1,continuous_update=False,layout=widgets.Layout(width='200px')),
    "wfh_uptake_percent":widgets.IntSlider(value=0,min=0,max=40,step=1,continuous_update=False,layout=widgets.Layout(width='200px')),
    "integrated_transport":widgets.Checkbox(),
    "bicycle_increase_percent":widgets.IntSlider(value=0,min=0,max=400,step=10,continuous_update=False,layout=widgets.Layout(width='200px'))
  }
  for rc in ["urban","rural"]:
    controls.update({
        f"parking_charge_{purpose}_{rc}":widgets.FloatSlider(value=0,min=0,max=20,step=0.5,continuous_update=False,layout=widgets.Layout(width='200px')) for purpose in purposes
    })
  controls.update({
      speed_param:widgets.FloatSlider(value=baseline,min=round(baseline)/2,max=baseline,step=0.5,continuous_update=False,layout=widgets.Layout(width='200px')) for speed_param,baseline in baseline_speeds_dict.items()
  })

  c = Struct(**controls)

  parking_widgets = {}
  for rc in ["urban","rural"]:
    parking_widgets[rc]=[]
    for purpose in purposes:
      parking_widgets[rc] += [widgets.Label(f"{purpose} £"),getattr(c,f"parking_charge_{purpose}_{rc}")]

  ui = widgets.VBox([
                      widgets.HTML(value="<b>Policy Levers:</b>"),
                      #widgets.HBox([c.length_filter, c.length_slice, widgets.Label("miles")]), # DO NOT REINSTATE WITHOUT CHECKING EVERY PLACE IT APPLIES
                      widgets.HBox([widgets.Label(f"Car cost per mile: PERCENT increase from baseline of £{baseline_car_fuel_costs_per_mile:.2f})"),c.car_cost_increase_percent]),
                      widgets.HBox([widgets.Label("Car average speed local (mph)"),c.speed_local,
                                    widgets.Label("local a-road urban"),c.speed_local_a_road_urban,
                                    widgets.Label("local a-road rural"),c.speed_local_a_road_rural,
                                    widgets.Label("trunk road"),c.speed_strategic]),
                      widgets.HBox([widgets.Label("Car congestion charge £"),c.congestion_charge,widgets.Label("applying to all settlements equal to or larger than"),c.most_rural_cc_category]),
                      widgets.HBox([widgets.Label("Parking charges Urban:")]+parking_widgets["urban"]),
                      widgets.HBox([widgets.Label("Parking charges Rural:")]+parking_widgets["rural"]),
                      widgets.HBox([widgets.Label("Rail fare discount %"),c.rail_fare_discount_percent,
                                  widgets.Label("Rail frequency increase %"),c.rail_frequency_increase_percent]),
                      widgets.HBox([widgets.Label("Bus fare discount %"),c.bus_fare_discount_percent,
                                    widgets.Label("Bus frequency increase %"),c.bus_frequency_increase_percent,
                                    widgets.Label("Bus speed increase (priority measures) %"),c.bus_speed_increase_percent]),
                      widgets.HBox([widgets.Label("Bus fare cap £"),c.bus_fare_cap,
                                    widgets.Label("applying to"),c.bus_fare_cap_applies,widgets.Label("after any % discount above")]),
                      widgets.HBox([widgets.Label("Integrated bus/rail transport"),c.integrated_transport]),
                      widgets.HTML(value="<b>Top down assumptions:</b>"),
                      widgets.HBox([widgets.Label("Divide per-passenger-mile costs by occupancy on:"),c.adjust_bus_costs_for_occupancy,c.adjust_rail_costs_for_occupancy],
                                   layout=widgets.Layout(display='inline-flex',flex_flow='row wrap')),
                      widgets.HBox([widgets.Label("Workplaces offering telework %"),c.wfh_uptake_percent,
                                    widgets.Label("Minimum average car occupancy"),c.min_average_car_occupancy,
                                    widgets.Label("Unaccounted diversion from driver to passenger"),c.unaccounted_driver_to_passenger_proportion]),
                      widgets.HBox([widgets.Label("Car club uptake %"),c.car_club_uptake_percent,widgets.Label("applying to all settlements equal to or larger than"),c.most_rural_car_club_category]),
                      widgets.HBox([widgets.Label("E-bike lifestyle uptake urban %"),c.ebike_uptake_urban_percent,widgets.Label("E-bike lifestyle uptake rural %"),c.ebike_uptake_rural_percent,
                                    widgets.Label("Other cycle increase %"),c.bicycle_increase_percent]),
                      widgets.HTML(value="<b>Results:</b>")
                    ]
  )

  def run_model(**args):
    global global_baseline,global_result
    args["length_predicate"] = {"All trips":all_trips(),
                             "Trips over":length_over(args["length_slice"]),
                             "Trips under":length_under(args["length_slice"])} [args["length_filter"]]

    args["baseline"] = global_baseline
    res = scenario_combined_activity_model(args)
    result = res.result
    global_result = result # for debug access only

    scenario_mode_totals = result.groupby("Mode").sum(numeric_only=True)
    baseline_mode_totals = global_baseline.groupby("Mode").sum(numeric_only=True)
    balance = scenario_mode_totals["Revenue"].sum()+scenario_mode_totals["Expenditure"].sum()
    absolute_car_reduction = baseline_mode_totals.loc["Car","model distance py"]-scenario_mode_totals.loc["Car","scenario distance py"]
    balance_per_mile_reduced = f"£{balance/absolute_car_reduction:.2f}" if absolute_car_reduction>0 else "N/A"

    bus_occupancy_multiplier = scenario_mode_totals.loc["Bus","scenario distance py"]/baseline_mode_totals.loc["Bus","model distance py"]/args["bus_frequency_increase"]
    rail_occupancy_multiplier = scenario_mode_totals.loc["Rail","scenario distance py"]/baseline_mode_totals.loc["Rail","model distance py"]/args["rail_frequency_increase"]

    mode_by_dist = result.groupby(["Mode","Distance"]).sum(numeric_only=True).unstack(0)["scenario distance py"]
    car_occupancy = (mode_by_dist["Car"]+mode_by_dist["Car passenger"])/mode_by_dist["Car"]
    min_car_occupancy = car_occupancy.min()
    min_car_occupancy_dist = car_occupancy[car_occupancy==car_occupancy.min()]
    car_occ_explanation = f" (for {min_car_occupancy_dist.index[0]} mile band)" if car_occupancy.max()-car_occupancy.min()>=0.01 else ""
    car_occupancy_mean = (scenario_mode_totals.loc["Car passenger","scenario distance py"]+scenario_mode_totals.loc["Car","scenario distance py"])/scenario_mode_totals.loc["Car","scenario distance py"]

    warnings=""
    if res.warnings!=[]:
      for w in res.warnings:
        warnings+='<p><font color="red"><b>WARNING: '+w+'</b></font></p>'

    HTML_OUTPUT = warnings+'<style>table.mode_table, .mode_table td { border: 1px solid grey; border-collapse: collapse; padding: 5px; }</style>'\
                  '<font size=3><table class="mode_table">'\
                  '<tr><td>Mode</td><td colspan=2>Miles per annum</td><td>Expenditure per mile</td><td>Expenditure</td><td>Revenue per mile</td><td>Revenue</td><td>Balance</td></tr>'

    scenario_mode_totals.loc["Total"] = scenario_mode_totals.sum()
    baseline_mode_totals.loc["Total"] = baseline_mode_totals.sum()

    for mode in ["Car","Car passenger","Rail","Bus","Walk","Bicycle","Total"]:
      mode_revenue = scenario_mode_totals.loc[mode,"Revenue"]
      mode_expenditure = scenario_mode_totals.loc[mode,"Expenditure"]
      scenario_miles = scenario_mode_totals.loc[mode,"scenario distance py"]
      baseline_miles = baseline_mode_totals.loc[mode,"model distance py"]
      mode_revenue_per_mile = mode_revenue/scenario_miles
      mode_expenditure_per_mile = mode_expenditure/scenario_miles
      percent_change = (scenario_miles-baseline_miles)/baseline_miles*100
      change_text = "no change" if percent_change==0 else f"{percent_change:.0f}% increase" if percent_change>0 else f"{abs(percent_change):.0f}% reduction"
      tr_style = ""
      total_style = 'style="font-weight:bold; font-style:italic"'
      if mode=="Total":
        tr_style = total_style
      if mode=="Car":
        change_text = f"<b>{change_text}</b>"
        mode_text = "Car driver"
      else:
        mode_text = mode
      col = f"<font color={mode_to_colour[mode]}>"
      HTML_OUTPUT += f'<tr {tr_style}><td>{col}{mode_text}</font></td><td align=right>{col}{scenario_miles:,.0f}</font></td><td>{col}({change_text})</font></td>'\
                     f'<td align=right>{col}£{mode_expenditure_per_mile:,.2f}</font></td><td align=right>{col}£{mode_expenditure:,.0f}</font></td>'\
                     f'<td align=right>{col}£{mode_revenue_per_mile:,.2f}</font></td><td align=right>{col}£{mode_revenue:,.0f}</font></td><td align=right>{col}£{mode_expenditure+mode_revenue:,.0f}</font></td></tr>'

    HTML_OUTPUT += f'<tr><td colspan=5><font size=2><i>Car occupancy minimum {min_car_occupancy:.2f}{car_occ_explanation}, average {car_occupancy_mean:.2f}; PT occupancy change: bus {bus_occupancy_multiplier*100-100:.0f}%, rail {rail_occupancy_multiplier*100-100:.0f}%</i></font></td>'\
                   f'<td align=right colspan=2 {total_style}><i>Balance per car mile reduced:</i></td><td align=right {total_style}>{balance_per_mile_reduced}</td></tr>'
    HTML_OUTPUT += '</table></font>'

    display(HTML(HTML_OUTPUT))
    pkm_barplot(result,global_baseline,"Distance")
    pkm_barplot(result,global_baseline,"ruralcat")

    # plot journey time increase by distance/ruralcat
    car_time_df = result.loc[car_mode(result)].groupby(["Distance","ruralcat"]).first().unstack(1)["car time factor"]
    car_time_df.loc[:,:]*=100
    car_time_df.loc[:,:]-=100
    fig, ax = plt.subplots(1,1,figsize=(15,3))
    car_time_df.plot(y=ruralcats,ax=ax,kind="bar",color=[mode_to_colour[x] for x in car_time_df.columns])
    ax.set_ylim(ymin=0)
    ax.set_ylabel("Car journey time increase %")
    plt.show()

    # plot car extra charges by distance/ruralcat. nb this time just for drivers, where costs are charged
    stack_df = result.loc[result.Mode=="Car"].groupby(["Distance","ruralcat"]).mean(numeric_only=True).unstack(1)
    revenue_df = stack_df["scenario_trip_car_cost_per_vehicle"]/stack_df["baseline_trip_car_cost_per_vehicle"]
    revenue_df.loc[:,:]*=100
    revenue_df.loc[:,:]-=100
    fig, ax = plt.subplots(1,1,figsize=(15,3))
    revenue_df.plot(y=ruralcats,ax=ax,kind="bar",color=[mode_to_colour[x] for x in revenue_df.columns])
    ax.set_ylim(ymin=0)
    ax.set_ylabel("Car cost per trip increase %")
    plt.show()

    # capture config as string
    config = args.copy()
    del config["baseline"]
    del config["length_predicate"]
    #display(HTML(f"<p>Scenario parameters</p><p style=\"font-family:'Courier New';size:1\">{config}</p>"))
    #print(result[(result.Mode=="Bus")&(result.purpose=="Leisure")&(result.ruralcat==ruralcats[0])])

  out = widgets.interactive_output(run_model, controls)
  out.layout.height = '2000px'

  def reset():
      # dirty hack: wiggle slider to force recomputation
      old_cost_factor = c.car_cost_increase_percent.value
      wiggle_value = 100 if old_cost_factor==200 else 200
      c.car_cost_increase_percent.value = wiggle_value
      c.car_cost_increase_percent.value = old_cost_factor

  return ui,out,reset

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

la_options = list(urban_rural_mix_table.index)
la_options.remove("England and Wales")
la_options.sort()
la_options.insert(0,"England and Wales")
la_dropdown = widgets.Dropdown(options=la_options)

ui, output, reset = create_gui()

def select_la(local_authority):
  load_global(local_authority)
  reset()

  def plots(df,popmix):
    fig, ax = plt.subplots(1,2,figsize=(15,3))
    popmix.plot.pie(ax = ax[0],colors=[mode_to_colour[x] for x in popmix.index],title="Population by rural category")
    df /= 1e9
    if df.columns.nlevels==2:
      df = df.droplevel(0,axis=1)
    for c in df.columns:
      if df[c].sum()==0:
        del df[c]
    df = df[df.sum(axis=1)>0]
    df.plot.bar(ax=ax[1],stacked=True,color=[mode_to_colour[x] for x in df.columns])
    ax[1].set_ylabel("Passenger miles per annum (billions)")
    ax[1].set_xlabel("Trip length (miles)")
    ax[1].set_title("Baseline Travel Demand")
    ax[1].get_legend().remove()
    plt.show()

  popmix = urban_rural_mix_table.loc[local_authority]
  popmix.name=""

  plots(global_baseline.groupby(["Distance","ruralcat"]).sum(numeric_only=True).unstack(),popmix)

la_out = widgets.interactive_output(select_la,{"local_authority":la_dropdown})
display(widgets.HBox([widgets.Label("Assume urban/rural mix representative of"),la_dropdown]),la_out)

display(ui,output)


<IPython.core.display.Javascript object>

Output()

Output(layout=Layout(height='2000px'))